In [1]:
from vllm import LLM, SamplingParams

In [2]:
with open('long-context.txt', 'r') as f:
    input = f.read()

In [3]:
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)
llm = LLM(model="THUDM/chatglm3-6b", trust_remote_code=True)

INFO 07-19 20:24:14 llm_engine.py:169] Initializing an LLM engine (v0.5.0.post1) with config: model='THUDM/chatglm3-6b', speculative_config=None, tokenizer='THUDM/chatglm3-6b', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=THUDM/chatglm3-6b, use_v2_block_manager=False, enable_prefix_caching=False)


Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.


WARNING 07-19 20:24:14 tokenizer.py:126] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.
INFO 07-19 20:24:17 weight_utils.py:218] Using model weights format ['*.safetensors']
INFO 07-19 20:24:19 model_runner.py:255] Loading model weights took 11.6579 GB
INFO 07-19 20:24:21 gpu_executor.py:84] # GPU blocks: 19399, # CPU blocks: 9362
INFO 07-19 20:24:23 model_runner.py:924] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-19 20:24:23 model_runner.py:928] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-19 20:24:32 model_runner.py:1117] Graph capturing finished in 9 secs.


In [4]:
tokenizer = llm.get_tokenizer()
tokens = tokenizer.encode(input)
tokens += tokens
print(len(tokens))

10626


In [5]:
AVG = 5
prefill_list = []
for token_len in range(500, 8001, 500):
    avg_prefill = 0
    for i in range(AVG):
        _, lat_list = llm.generate(None, sampling_params, prompt_token_ids=tokens[:token_len])
        avg_prefill += lat_list[0]
    prefill_list.append(avg_prefill/AVG)
print(prefill_list)


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.65it/s, est. speed input: 1327.60 toks/s, output: 42.48 toks/s]


[91164.35050964355, 21018.505096435547, 20417.92869567871, 20416.259765625, 20400.524139404297, 20399.57046508789, 18862.009048461914, 18266.20101928711, 18197.53646850586, 18257.856369018555, 18193.24493408203, 18180.84716796875, 18191.099166870117, 18193.721771240234, 18146.038055419922, 18195.86753845215]


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.95it/s, est. speed input: 1478.57 toks/s, output: 47.31 toks/s]


[63842.058181762695, 18800.25863647461, 18623.59046936035, 18433.332443237305, 18327.951431274414, 18231.630325317383, 18194.198608398438, 18207.550048828125, 18171.310424804688, 18194.913864135742, 18201.112747192383, 18150.56800842285, 18133.163452148438, 18178.701400756836, 18177.270889282227, 18227.100372314453]


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.94it/s, est. speed input: 1471.56 toks/s, output: 47.09 toks/s]


...

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.08s/it, est. speed input: 5550.59 toks/s, output: 14.80 toks/s]


[803904.5333862305, 19165.992736816406, 18882.036209106445, 18553.733825683594, 18452.882766723633, 18335.342407226562, 18304.58641052246, 18349.170684814453, 18336.057662963867, 18321.27571105957, 18330.57403564453, 18294.334411621094, 18275.022506713867, 18471.479415893555, 18390.893936157227, 18507.24220275879]


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.08s/it, est. speed input: 5547.93 toks/s, output: 14.79 toks/s]


[804391.1457061768, 19195.556640625, 18873.929977416992, 18573.04573059082, 18490.076065063477, 18350.36277770996, 18348.93226623535, 18377.78091430664, 18335.10398864746, 18350.839614868164, 18347.501754760742, 18303.15589904785, 18302.440643310547, 18311.023712158203, 18372.058868408203, 18483.638763427734]


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]